In [3]:
import pandas as pd 
import warnings

In [4]:
# Remove red warning pop-up 
warnings.filterwarnings('ignore')

In [5]:
! ls ../archive

AviationData.csv           Untitled.ipynb
NEW_cleaned_aviation.csv   cleaned_aviation_data.csv
USState_Codes.csv          cleaned_aviation_data1.csv


In [6]:
# Read in data file, ensuring the encoding is set so it doesn't throw an error
df = pd.read_csv('../archive/AviationData.csv', encoding = 'latin1')

In [7]:
# Check out Data
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [8]:
# View the number of columns, as well as their data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

In [9]:
# View the number of NaNs for each column.
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38640
Airport.Name              36099
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1317
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7077
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6381
Publication.Date          13771
dtype: i

In [10]:
# Calculate the percentage of values being NaN for each column
rows = len(df)
missing = df.isna().sum()
percentage_missing = missing / rows

In [11]:
# Put the data in a DataFrame and sort it
percentage_missing_df = pd.DataFrame({'Missing' : percentage_missing})
percentage_missing_df.sort_values('Missing', ascending = False, inplace = True)

In [12]:
#Show the columns with the most missing data
print(percentage_missing_df[percentage_missing_df['Missing'] > 0.1])

                         Missing
Schedule                0.858453
Air.carrier             0.812710
FAR.Description         0.639742
Aircraft.Category       0.636772
Longitude               0.613304
Latitude                0.613203
Airport.Code            0.434699
Airport.Name            0.406113
Broad.phase.of.flight   0.305606
Publication.Date        0.154924
Total.Serious.Injuries  0.140737
Total.Minor.Injuries    0.134246
Total.Fatal.Injuries    0.128261


In [13]:
# Drop columns with over 50% missing values, Print the na
cols_to_drop = list(percentage_missing_df[percentage_missing_df['Missing'] > 0.5].index)
df.drop(columns = cols_to_drop, axis = 1, inplace = True)
print(cols_to_drop)

['Schedule', 'Air.carrier', 'FAR.Description', 'Aircraft.Category', 'Longitude', 'Latitude']


In [14]:
# Drop records that are neither Accidents nor take place in the United States
before = len(df)
df = df[(df['Investigation.Type'] == 'Accident') & (df['Country'] == 'United States')]
dropped = before - len(df)
print(str(dropped) + ' rows dropped.')

8983 rows dropped.


In [15]:
# Convert Date to a datetime, view updated column data
df['Event.Date'] = pd.to_datetime(df['Event.Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79906 entries, 0 to 88888
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                79906 non-null  object        
 1   Investigation.Type      79906 non-null  object        
 2   Accident.Number         79906 non-null  object        
 3   Event.Date              79906 non-null  datetime64[ns]
 4   Location                79895 non-null  object        
 5   Country                 79906 non-null  object        
 6   Airport.Code            47566 non-null  object        
 7   Airport.Name            50000 non-null  object        
 8   Injury.Severity         79854 non-null  object        
 9   Aircraft.damage         78782 non-null  object        
 10  Registration.Number     79903 non-null  object        
 11  Make                    79894 non-null  object        
 12  Model                   79877 non-null  object

In [16]:
#Add a day, month & year column
df['Year'] = df['Event.Date'].dt.year
df['Month.Abbr'] = df['Event.Date'].dt.month_name().str[:3]
df['Day.Name.Abbr'] = df['Event.Date'].dt.day_name().str[:3]

# Remove data before 2000
df = df[df['Year'] >= 2000]

In [17]:
# Merge same airport names together
df['Airport.Name'].replace(to_replace = '(?i)^.*private.*$', value = 'PRIVATE', inplace = True, regex = True)
df['Airport.Name'].replace(to_replace = '(?i)^.*none.*$', value = 'NONE', inplace = True, regex = True)
df['Airport.Name'].value_counts()

PRIVATE                           671
NONE                              126
North Las Vegas                    40
PVT                                35
Centennial Airport                 27
                                 ... 
Charlotte                           1
Steve Berg Field                    1
Skyview                             1
Lakeland                            1
WICHITA DWIGHT D EISENHOWER NT      1
Name: Airport.Name, Length: 12745, dtype: int64

In [18]:
# Merge same registration numbers together
df['Registration.Number'].replace(to_replace = '(?i)^.*none.*$', value = 'NONE', inplace = True, regex = True)
df['Registration.Number'].value_counts()

NONE      119
UNREG     114
N5408Y      5
N8402K      5
N121CC      5
         ... 
N9709       1
N6134X      1
N1410C      1
N248PC      1
N9026P      1
Name: Registration.Number, Length: 32976, dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34904 entries, 47675 to 88888
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                34904 non-null  object        
 1   Investigation.Type      34904 non-null  object        
 2   Accident.Number         34904 non-null  object        
 3   Event.Date              34904 non-null  datetime64[ns]
 4   Location                34897 non-null  object        
 5   Country                 34904 non-null  object        
 6   Airport.Code            23475 non-null  object        
 7   Airport.Name            23765 non-null  object        
 8   Injury.Severity         34852 non-null  object        
 9   Aircraft.damage         34298 non-null  object        
 10  Registration.Number     34902 non-null  object        
 11  Make                    34899 non-null  object        
 12  Model                   34897 non-null  ob

In [20]:
# Merge different capitalizations of 'Make' together
df['Make'] = df['Make'].str.title()
df['Make'].value_counts()

Cessna            9460
Piper             5354
Beech             1965
Bell               889
Mooney             537
                  ... 
Degelia              1
Lambert John G       1
Indus                1
Dusenberry           1
Royse Ralph L        1
Name: Make, Length: 5043, dtype: int64

In [21]:
# Split location in city and state
df['City'] = df['Location'].str.split(',').str[0]
df['State'] = df['Location'].str.split(',').str[1]
df[['City', 'State']].head(10)

,City,State
47675,HOMESTEAD,FL
47676,MONTEAGLE,TN
47677,VICTORVILLE,CA
47678,DOS PALOS,CA
47679,CORNING,AR
47680,ODESSA,TX
47681,TELLURIDE,CO
47684,HOUSTON,TX
47685,CORDESVILLE,SC
47686,KISSIMMEE,FL


In [22]:
# Remove amount of injuries as this is aleady in another column
df['Injury.Severity'] = df['Injury.Severity'].str.split('(').str[0]
df['Injury.Severity'].value_counts()

Non-Fatal    28168
Fatal         6328
Minor          203
Serious        153
Name: Injury.Severity, dtype: int64

In [23]:
# Merge weather condition unknowns
df['Weather.Condition'].replace(to_replace = ['Unk', 'UNK'], value = 'Unknown', inplace = True)
df['Weather.Condition'].value_counts()

VMC        32401
IMC         1843
Unknown      106
Name: Weather.Condition, dtype: int64

In [24]:
# Dropping the event ID column
df = df.drop(columns= 'Event.Id')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34904 entries, 47675 to 88888
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Investigation.Type      34904 non-null  object        
 1   Accident.Number         34904 non-null  object        
 2   Event.Date              34904 non-null  datetime64[ns]
 3   Location                34897 non-null  object        
 4   Country                 34904 non-null  object        
 5   Airport.Code            23475 non-null  object        
 6   Airport.Name            23765 non-null  object        
 7   Injury.Severity         34852 non-null  object        
 8   Aircraft.damage         34298 non-null  object        
 9   Registration.Number     34902 non-null  object        
 10  Make                    34899 non-null  object        
 11  Model                   34897 non-null  object        
 12  Amateur.Built           34890 non-null  ob

In [26]:
#Dropping all rows that contain amateur built planes, then dropping the 'Amateur.Built' column entirely
df = df[df['Amateur.Built'] != 'Yes']
df = df.drop(columns= 'Amateur.Built')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30346 entries, 47675 to 88888
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Investigation.Type      30346 non-null  object        
 1   Accident.Number         30346 non-null  object        
 2   Event.Date              30346 non-null  datetime64[ns]
 3   Location                30339 non-null  object        
 4   Country                 30346 non-null  object        
 5   Airport.Code            20352 non-null  object        
 6   Airport.Name            20598 non-null  object        
 7   Injury.Severity         30294 non-null  object        
 8   Aircraft.damage         29741 non-null  object        
 9   Registration.Number     30344 non-null  object        
 10  Make                    30345 non-null  object        
 11  Model                   30341 non-null  object        
 12  Number.of.Engines       29456 non-null  fl

In [29]:
df['Total.Passengers'] = df['Total.Fatal.Injuries'] + df['Total.Serious.Injuries'] + df['Total.Minor.Injuries'] + df['Total.Uninjured']

df['Total.Passengers'].value_counts()

1.0      8984
2.0      6747
3.0      1632
4.0       960
5.0       252
         ... 
251.0       1
93.0        1
165.0       1
222.0       1
203.0       1
Name: Total.Passengers, Length: 200, dtype: int64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30346 entries, 47675 to 88888
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Investigation.Type      30346 non-null  object        
 1   Accident.Number         30346 non-null  object        
 2   Event.Date              30346 non-null  datetime64[ns]
 3   Location                30339 non-null  object        
 4   Country                 30346 non-null  object        
 5   Airport.Code            20352 non-null  object        
 6   Airport.Name            20598 non-null  object        
 7   Injury.Severity         30294 non-null  object        
 8   Aircraft.damage         29741 non-null  object        
 9   Registration.Number     30344 non-null  object        
 10  Make                    30345 non-null  object        
 11  Model                   30341 non-null  object        
 12  Number.of.Engines       29456 non-null  fl